In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import numpy as np
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape,Conv2D,MaxPool2D,Flatten,Activation
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
# The path of the saving model check points
save_check_pt = './checkpoints_DANN'

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = cv2.imread(img_dir)
        img = cv2.resize(img,(16,32))
        X.append(img)
    X = np.array(X)
    return X

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = Image.open(img_dir)
#         img = img.convert('L') # conver to grayscale images
#         img = img.resize([16, 32])
#         img_np = np.asarray(img)
#         X.append(img_np.reshape([-1]))
#     X = np.array(X)
#     return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(32,16,3)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[3, 3], padding="same", activation='relu',input_shape=input_shape))
    model.add(MaxPool2D(pool_size=[2, 2], strides=2, padding='same'))
    model.add(Conv2D(48, kernel_size=[3, 3], padding="same", activation='relu'))
    model.add(MaxPool2D(pool_size=[2, 2], strides=2, padding='same'))
    model.add(Flatten())
    model.add(Activation('sigmoid'))
    return model

def get_task(input_shape=(1536,)):
    model = Sequential()
    model.add(Dense(1024, activation='relu',input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(1536,)):
    model = Sequential()
    model.add(Dense(1024, activation='relu',input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=34)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=34)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=34)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=34)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=34)
db_images,db_labels = get_img_label('db', 'all', num_classes=34)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=34)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=34)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=34)
print(np.shape(base_images))
#model = get_encoder()
#model.build(input_shape=[None, 32, 16, 3])
#model.summary()
print("load data finished!")

(8446, 32, 16, 3)
load data finished!


In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(weather_images_t))
print(np.shape(weather_labels_t))
print(np.shape(weather_images))
print(np.shape(weather_labels))

weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(base_images, base_labels,weather_images_t,weather_labels_t, epochs=86,verbose=1,
                  batch_size = 32,validation_data=(weather_images, weather_labels))
weather_model.score(base_images, base_labels)
weather_model.score(weather_images, weather_labels)
weather_model.save_weights(save_check_pt+ '/weather')

(8446, 32, 16, 3)
(8446, 34)
(762, 32, 16, 3)
(762, 34)
(3812, 32, 16, 3)
(3812, 34)
Epoch 1/86
286/286 [==============================] - 3s 7ms/step - loss: 2.4887 - acc: 0.3724 - disc_loss: 1.1613 - disc_acc: 0.7014 - val_loss: 2.1610 - val_acc: 0.3927
Epoch 2/86
286/286 [==============================] - 2s 6ms/step - loss: 1.5385 - acc: 0.6183 - disc_loss: 1.0674 - disc_acc: 0.7355 - val_loss: 1.3717 - val_acc: 0.6645
Epoch 3/86
286/286 [==============================] - 2s 7ms/step - loss: 1.0035 - acc: 0.7410 - disc_loss: 1.0167 - disc_acc: 0.7487 - val_loss: 0.9908 - val_acc: 0.7411
Epoch 4/86
286/286 [==============================] - 2s 6ms/step - loss: 0.7682 - acc: 0.7957 - disc_loss: 0.9793 - disc_acc: 0.7625 - val_loss: 0.8267 - val_acc: 0.7878
Epoch 5/86
286/286 [==============================] - 2s 6ms/step - loss: 0.6273 - acc: 0.8362 - disc_loss: 0.9544 - disc_acc: 0.7678 - val_loss: 0.7075 - val_acc: 0.8111
Epoch 6/86
286/286 [==============================] - 2s 6ms

286/286 [==============================] - 2s 7ms/step - loss: 0.0543 - acc: 0.9890 - disc_loss: 0.3638 - disc_acc: 0.9286 - val_loss: 0.2743 - val_acc: 0.9349
Epoch 49/86
286/286 [==============================] - 2s 7ms/step - loss: 0.0554 - acc: 0.9891 - disc_loss: 0.3670 - disc_acc: 0.9306 - val_loss: 0.2202 - val_acc: 0.9541
Epoch 50/86
286/286 [==============================] - 2s 7ms/step - loss: 0.0618 - acc: 0.9872 - disc_loss: 0.4093 - disc_acc: 0.9197 - val_loss: 0.2469 - val_acc: 0.9449
Epoch 51/86
286/286 [==============================] - 2s 7ms/step - loss: 0.0552 - acc: 0.9878 - disc_loss: 0.3956 - disc_acc: 0.9219 - val_loss: 0.2547 - val_acc: 0.9481
Epoch 52/86
286/286 [==============================] - 2s 6ms/step - loss: 0.0552 - acc: 0.9885 - disc_loss: 0.3853 - disc_acc: 0.9235 - val_loss: 0.2484 - val_acc: 0.9478
Epoch 53/86
286/286 [==============================] - 2s 6ms/step - loss: 0.0568 - acc: 0.9890 - disc_loss: 0.4078 - disc_acc: 0.9167 - val_loss: 0.236

In [19]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(challenge_images_t))
print(np.shape(challenge_labels_t))
print(np.shape(challenge_images))
print(np.shape(challenge_labels))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(base_images, base_labels,challenge_images_t,challenge_labels_t,
                    epochs=60,verbose=1,batch_size = 32,validation_data=(challenge_images, challenge_labels))
challenge_model.score(base_images, base_labels)
challenge_model.score(challenge_images, challenge_labels)
challenge_model.save_weights(save_check_pt+ '/challenge')

(8446, 32, 16, 3)
(8446, 34)
(109, 32, 16, 3)
(109, 34)
(548, 32, 16, 3)
(548, 34)
Epoch 1/60
266/266 [==============================] - 3s 7ms/step - loss: 2.5330 - acc: 0.3594 - disc_loss: 0.8592 - disc_acc: 0.7959 - val_loss: 2.5350 - val_acc: 0.2828
Epoch 2/60
266/266 [==============================] - 2s 6ms/step - loss: 1.6302 - acc: 0.5981 - disc_loss: 0.6473 - disc_acc: 0.8600 - val_loss: 2.0770 - val_acc: 0.4434
Epoch 3/60
266/266 [==============================] - 2s 6ms/step - loss: 1.1359 - acc: 0.7069 - disc_loss: 0.5486 - disc_acc: 0.8911 - val_loss: 1.7939 - val_acc: 0.5091
Epoch 4/60
266/266 [==============================] - 2s 6ms/step - loss: 0.8824 - acc: 0.7669 - disc_loss: 0.4726 - disc_acc: 0.9078 - val_loss: 1.7020 - val_acc: 0.5401
Epoch 5/60
266/266 [==============================] - 2s 6ms/step - loss: 0.7337 - acc: 0.8085 - disc_loss: 0.3943 - disc_acc: 0.9272 - val_loss: 1.4442 - val_acc: 0.6624
Epoch 6/60
266/266 [==============================] - 2s 6ms/s

266/266 [==============================] - 2s 6ms/step - loss: 0.0574 - acc: 0.9891 - disc_loss: 0.0767 - disc_acc: 0.9879 - val_loss: 1.0937 - val_acc: 0.7883
Epoch 49/60
266/266 [==============================] - 2s 6ms/step - loss: 0.0596 - acc: 0.9889 - disc_loss: 0.0213 - disc_acc: 0.9974 - val_loss: 1.0284 - val_acc: 0.8212
Epoch 50/60
266/266 [==============================] - 2s 6ms/step - loss: 0.0448 - acc: 0.9929 - disc_loss: 0.0411 - disc_acc: 0.9941 - val_loss: 1.1241 - val_acc: 0.8102
Epoch 51/60
266/266 [==============================] - 2s 6ms/step - loss: 0.0490 - acc: 0.9904 - disc_loss: 0.0525 - disc_acc: 0.9919 - val_loss: 1.2506 - val_acc: 0.7482
Epoch 52/60
266/266 [==============================] - 2s 6ms/step - loss: 0.0652 - acc: 0.9858 - disc_loss: 0.0538 - disc_acc: 0.9914 - val_loss: 1.0243 - val_acc: 0.8029
Epoch 53/60
266/266 [==============================] - 2s 6ms/step - loss: 0.0547 - acc: 0.9893 - disc_loss: 0.0356 - disc_acc: 0.9947 - val_loss: 1.187

In [17]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(db_images_t))
print(np.shape(db_labels_t))
print(np.shape(db_images))
print(np.shape(db_labels))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(base_images, base_labels,db_images_t,db_labels_t, epochs=75,
             verbose=1,batch_size = 32,validation_data=(db_images, db_labels))
db_model.score(base_images, base_labels)
db_model.score(db_images, db_labels)
db_model.save_weights(save_check_pt+ '/db')

(8446, 32, 16, 3)
(8446, 34)
(373, 32, 16, 3)
(373, 34)
(1869, 32, 16, 3)
(1869, 34)
Epoch 1/75
269/269 [==============================] - 3s 7ms/step - loss: 2.5443 - acc: 0.3537 - disc_loss: 0.8921 - disc_acc: 0.8052 - val_loss: 2.2829 - val_acc: 0.4452
Epoch 2/75
269/269 [==============================] - 2s 6ms/step - loss: 1.6853 - acc: 0.5856 - disc_loss: 0.9085 - disc_acc: 0.7903 - val_loss: 1.7879 - val_acc: 0.5543
Epoch 3/75
269/269 [==============================] - 2s 7ms/step - loss: 1.1820 - acc: 0.6931 - disc_loss: 0.9568 - disc_acc: 0.7736 - val_loss: 1.5569 - val_acc: 0.5805
Epoch 4/75
269/269 [==============================] - 2s 6ms/step - loss: 0.9379 - acc: 0.7497 - disc_loss: 0.9612 - disc_acc: 0.7730 - val_loss: 1.2984 - val_acc: 0.6790
Epoch 5/75
269/269 [==============================] - 2s 6ms/step - loss: 0.7875 - acc: 0.7876 - disc_loss: 0.9423 - disc_acc: 0.7747 - val_loss: 1.2078 - val_acc: 0.6891
Epoch 6/75
269/269 [==============================] - 2s 6ms

269/269 [==============================] - 2s 6ms/step - loss: 0.0958 - acc: 0.9772 - disc_loss: 0.3753 - disc_acc: 0.9259 - val_loss: 0.6320 - val_acc: 0.8705
Epoch 49/75
269/269 [==============================] - 2s 6ms/step - loss: 0.0943 - acc: 0.9803 - disc_loss: 0.3232 - disc_acc: 0.9389 - val_loss: 0.6650 - val_acc: 0.8555
Epoch 50/75
269/269 [==============================] - 2s 7ms/step - loss: 0.0931 - acc: 0.9788 - disc_loss: 0.3047 - disc_acc: 0.9423 - val_loss: 0.7107 - val_acc: 0.8448
Epoch 51/75
269/269 [==============================] - 2s 6ms/step - loss: 0.1115 - acc: 0.9712 - disc_loss: 0.3184 - disc_acc: 0.9374 - val_loss: 0.6901 - val_acc: 0.8700
Epoch 52/75
269/269 [==============================] - 2s 6ms/step - loss: 0.0870 - acc: 0.9800 - disc_loss: 0.3450 - disc_acc: 0.9305 - val_loss: 0.5668 - val_acc: 0.8801
Epoch 53/75
269/269 [==============================] - 2s 6ms/step - loss: 0.0868 - acc: 0.9808 - disc_loss: 0.3073 - disc_acc: 0.9422 - val_loss: 0.611

In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(fn_images_t))
print(np.shape(fn_labels_t))
print(np.shape(fn_images))
print(np.shape(fn_labels))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(base_images, base_labels,fn_images_t,fn_labels_t, epochs=60,verbose=1,
             batch_size = 32,validation_data=(fn_images, fn_labels))
fn_model.score(base_images, base_labels)
fn_model.score(fn_images, fn_labels)
fn_model.save_weights(save_check_pt+ '/fn')

(8446, 32, 16, 3)
(8446, 34)
(48, 32, 16, 3)
(48, 34)
(243, 32, 16, 3)
(243, 34)
Epoch 1/60
264/264 [==============================] - 3s 7ms/step - loss: 2.5634 - acc: 0.3475 - disc_loss: 0.9835 - disc_acc: 0.7653 - val_loss: 2.2690 - val_acc: 0.4527
Epoch 2/60
264/264 [==============================] - 2s 6ms/step - loss: 1.6982 - acc: 0.5868 - disc_loss: 0.7798 - disc_acc: 0.8255 - val_loss: 1.5979 - val_acc: 0.5679
Epoch 3/60
264/264 [==============================] - 2s 6ms/step - loss: 1.1712 - acc: 0.6941 - disc_loss: 0.6733 - disc_acc: 0.8565 - val_loss: 1.2043 - val_acc: 0.6708
Epoch 4/60
264/264 [==============================] - 2s 6ms/step - loss: 0.9244 - acc: 0.7564 - disc_loss: 0.5387 - disc_acc: 0.8946 - val_loss: 1.0022 - val_acc: 0.7037
Epoch 5/60
264/264 [==============================] - 2s 6ms/step - loss: 0.7720 - acc: 0.7920 - disc_loss: 0.4310 - disc_acc: 0.9208 - val_loss: 0.7826 - val_acc: 0.8066
Epoch 6/60
264/264 [==============================] - 2s 6ms/ste

264/264 [==============================] - 2s 6ms/step - loss: 0.0485 - acc: 0.9909 - disc_loss: 0.0516 - disc_acc: 0.9926 - val_loss: 0.2044 - val_acc: 0.9506
Epoch 49/60
264/264 [==============================] - 2s 6ms/step - loss: 0.0515 - acc: 0.9905 - disc_loss: 0.0144 - disc_acc: 0.9981 - val_loss: 0.1014 - val_acc: 0.9630
Epoch 50/60
264/264 [==============================] - 2s 6ms/step - loss: 0.0506 - acc: 0.9905 - disc_loss: 0.0150 - disc_acc: 0.9977 - val_loss: 0.1982 - val_acc: 0.9547
Epoch 51/60
264/264 [==============================] - 2s 6ms/step - loss: 0.0507 - acc: 0.9899 - disc_loss: 0.0151 - disc_acc: 0.9980 - val_loss: 0.1060 - val_acc: 0.9712
Epoch 52/60
264/264 [==============================] - 2s 6ms/step - loss: 0.0371 - acc: 0.9938 - disc_loss: 0.0042 - disc_acc: 0.9995 - val_loss: 0.0967 - val_acc: 0.9753
Epoch 53/60
264/264 [==============================] - 2s 6ms/step - loss: 0.0691 - acc: 0.9847 - disc_loss: 0.0780 - disc_acc: 0.9883 - val_loss: 0.132